# Code Generator

So it turns out that neo4j does not support this kind of comparison as a UDF;

```java
@UserFunction
@Description("engineernick.anydatemin(dateA, dateB) - returns the lesser of any two comparable objects.")
public static <T extends Comparable<T>> T min(
    @Name("dateA") T dateA,
    @Name("dateB") T dateB
) {
    return dateA.compareTo(dateB) <= 0 ? dateA : dateB;
}
```

However it does recognise the following types `[boolean, byte[], double, java.lang.Boolean, java.lang.Double, java.lang.Long, java.lang.Number, java.lang.Object, java.lang.String, java.time.LocalDate, java.time.LocalDateTime, java.time.LocalTime, java.time.OffsetTime, java.time.ZonedDateTime, java.time.temporal.TemporalAmount, java.util.List, java.util.Map, long, org.neo4j.graphdb.Node, org.neo4j.graphdb.Path, org.neo4j.graphdb.Relationship, org.neo4j.graphdb.spatial.Geometry, org.neo4j.graphdb.spatial.Point]`

And since we don't have a nice macro system, its time for code generation!

The values we care most about are the basic datetime values

In [ ]:
import itertools
from pathlib import Path
from typing import Literal

In [ ]:
java_date_types = [
    {
        "type":"java.time.LocalDate"    ,
        "name_suffix":"date",          
        "example_small":"2022-01-01",
        "example_large":"2023-01-01",
    },
    {
        "type":"java.time.LocalDateTime",
        "name_suffix":"localdatetime", 
        "example_small":"2022-01-01T16:30:00",
        "example_large":"2023-01-01T17:30:00",
    },
    {
        "type":"java.time.ZonedDateTime",
        "name_suffix":"datetime",      
        "example_small":"2022-01-01T16:30:00+08:00",
        "example_large":"2023-01-01T17:30:00+08:00",
    },
]

In [ ]:
def create_function_and_test(
    package_name:str,
    method:dict,
    type_a:dict,
    type_b:dict,
    operator:Literal["<=",">="],
):
    # currently priority does not matter
    return_type = type_a["type"]
    function_name = f'{method["name"]}_{type_a["name_suffix"]}'
    a_example_small = f"""{type_a["name_suffix"]}('{type_a["example_small"]}')"""
    a_example_large = f"""{type_a["name_suffix"]}('{type_a["example_large"]}')"""
    expected_result = type_a["example_small"] if method["op"]=="<=" else type_a["example_large"]
    full_op_name = {
        "min":"minimum",
        "max":"maximum"
    }[method["name"]]
    return {
        "method":f"""
    @UserFunction
    @Description("{package_name}.{function_name}(dateA, dateB) - returns the {full_op_name} of two {type_a["name_suffix"]} objects, or the first non-null argument.")
    public {return_type} {function_name}(
        @Name("dateA") {type_a["type"]} dateA,
        @Name("dateB") {type_b["type"]} dateB
    ) {{
        if (dateA == null) {{
            return (dateB == null) ? null : dateB;
        }}
        if (dateB == null) {{
            return dateA;
        }}
        return dateA.compareTo(dateB) {operator} 0 ? dateA : dateB;
    }}""",
        "test":f"""
    @Test
    void {function_name}_returns_expected() {{
        try (Driver driver = GraphDatabase.driver(embeddedDatabaseServer.boltURI());
             Session session = driver.session()) {{
            assertThat(
                session.run(\"\"\"
                    RETURN toString(
                        {package_name}.{function_name}( {a_example_small}, {a_example_large} )
                    )
                \"\"\").single().get(0).asString()
            ).isEqualTo("{expected_result}");
        }}
    }}"""
    }

In [ ]:
results = []
for date_type, method in itertools.product(
        java_date_types,
        [
            {"name":"min", "op":"<="},
            {"name":"max", "op":">="},
        ]
    ):
    results.append(
        create_function_and_test(
            package_name="ndt",
            method=method,
            type_a=date_type,
            type_b=date_type,
            operator=method["op"],
        )
    )
class_methods = "\n".join(item["method"] for item in results)
class_test_methods = "\n".join(item["test"] for item in results)

In [ ]:
class_name = "DateMinMaxGen"
class_name_test = "DateMinMaxGenTest"
package_name = "ndt"

outfile = Path(f"./src/main/java/org/neo4j/{package_name}/{class_name}.java")
outfile.parent.mkdir(parents=True, exist_ok=True)

outfile.write_text(f"""
package {package_name};

import java.util.List;

import org.neo4j.procedure.Description;
import org.neo4j.procedure.Name;
import org.neo4j.procedure.UserFunction;

import java.time.LocalDate;
import java.time.LocalDateTime;

public class {class_name} {{
    {class_methods}
}}
""")

outfile_test = Path(f"./src/test/java/org/neo4j/{package_name}/{class_name_test}.java")
outfile_test.parent.mkdir(parents=True, exist_ok=True)

outfile_test.write_text(f"""
package {package_name};

import org.junit.jupiter.api.AfterAll;
import org.junit.jupiter.api.BeforeAll;
import org.junit.jupiter.api.Test;
import org.junit.jupiter.api.TestInstance;
import org.neo4j.driver.Driver;
import org.neo4j.driver.GraphDatabase;
import org.neo4j.driver.Session;
import org.neo4j.harness.Neo4j;
import org.neo4j.harness.Neo4jBuilders;

import java.time.LocalDate;
import java.time.LocalDateTime;

import org.neo4j.driver.Value;
import static org.assertj.core.api.Assertions.assertThat;

@TestInstance(TestInstance.Lifecycle.PER_CLASS)
public class {class_name_test} {{

    private Neo4j embeddedDatabaseServer;

    @BeforeAll
    void initializeNeo4j() {{
        this.embeddedDatabaseServer = Neo4jBuilders.newInProcessBuilder()
                .withDisabledServer()
                .withFunction({class_name}.class)
                .build();
    }}

    @AfterAll
    void closeNeo4j() {{
        this.embeddedDatabaseServer.close();
    }}

    {class_test_methods}
}}
""")